In [ ]:
#引入需要的包
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from time import time
from __future__ import division

%matplotlib inline

In [ ]:
#定义单张图片dct特征的提取方法
def dct(img):
    h,w,_ = img.shape
    assert h >= 8 and w >= 8,'请使用大小至少为8x8的图片'
    img = np.float64(img)/255.0
    out = np.zeros((1,3,8,8),dtype=np.float64)
    N = (h // 2 - 3) * (w // 2 - 3)
    for i in xrange(h // 2 - 3):
        for j in xrange(w // 2 - 3):
            r = img[2*i:2*i+8,2*j:2*j+8,0]
            g = img[2*i:2*i+8,2*j:2*j+8,1]
            b = img[2*i:2*i+8,2*j:2*j+8,2]
            r_ = cv2.dct(r).reshape(1,1,8,8)
            g_ = cv2.dct(g).reshape(1,1,8,8)
            b_ = cv2.dct(b).reshape(1,1,8,8)
            temp = np.concatenate((r_,g_,b_),axis = 1)
            out = np.concatenate((out,temp),axis=0)
    out = out[1:]
    return out,N

#特征降维（从192降至63）
def down_size(img,N):
    out = img.reshape(N,3,-1)[:,:,[0,1,2,3,4,5,8,9,10,11,12,16,17,18,19,24,25,26,32,33,40]].reshape(N,-1)
    return out

In [ ]:
#定义测试图片的方法
def gaussion_prob(data_U,U,E,th=5e-4):
    N,dim = data_U.shape
    m = U.shape[0]
    E_d = np.eye(dim)*th
    prob = np.zeros((m,N))
    inv_Es = np.zeros((m,dim,dim))
    
    for i in range(m):
        det_E = np.linalg.det(E[i]+E_d)
        inv_E = np.linalg.inv(E[i]+E_d)
        inv_Es[i] = inv_E
        xishu = 1 / (det_E ** 0.5 * (2.0 * np.pi) ** (dim/2))
        X_U = data_U - U[i]
        zhishu = np.sum(np.dot(X_U,inv_E) * X_U,axis=1) * (-0.5)
        prob[i] = xishu * np.exp(zhishu)
        
    return prob,inv_Es

def gmm_prob(x,U,E,A):
    prob,_ = gaussion_prob(x,U,E)
    prob *= A[:,np.newaxis]
    prob = np.sum(prob,axis=0)
    return prob
    
def test(img,mods):
    img_dct,N = dct(img)
    img_down = down_size(img_dct,N)
    Us,Es,As = mods['U'],mods['E'],mods['A']
    num = Us.shape[0]
    model_probs = np.zeros(num)
    for i in range(num):
        model_probs[i] = np.sum(np.log(gmm_prob(img_down,Us[i],Es[i],As[i])))
    model_probs += np.log(label_times)#4500没必要除
    img_labels = np.argsort(model_probs)[::-1]
    #print model_probs
    return img_labels

In [ ]:
#读取模型
models = np.load('models.npz')

#读取每种标签在训练集中出现的次数
label_times = np.load('label_times.npy')

#读取标签对应的文本，显示前五个
label_word = np.loadtxt('new_word.txt',dtype=np.str)
print label_word.shape
print label_word[:5]

In [ ]:
#读取，标注，显示测试图片。
tic = time()
img = mpimg.imread('test.jpeg')#将test.jpeg改为任意图片（任何size都行，但最小不能小于8x8，太大的话运行会很慢，要resize的话请提前做好）
annotation = test(img,models)
print '共花费',time() - tic,'秒'
plt.imshow(img)
label_word[annotation][:5]#自行设置标注数目，越靠前语义越接近